In [1]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 10)

import warnings
warnings.filterwarnings('ignore')

# I. Import data

In [2]:
label = pd.read_parquet(r'data/label.parquet')
cus_level = pd.read_parquet('data/cus_level.parquet')
root = pd.read_parquet('data/root_level.parquet')

contract_install = pd.read_parquet('data/gr_ins.parquet')
contract_noninstall = pd.read_parquet('data/gr_nonins.parquet')
contract_card = pd.read_parquet('data/gr_card.parquet')

not_grant_install = pd.read_parquet('data/ng_ins.parquet')
not_grant_noninstall = pd.read_parquet('data/ng_nonins.parquet')
not_grant_card = pd.read_parquet('data/ng_card.parquet')

ts_install = pd.read_parquet('data/ts_install.parquet')
ts_card = pd.read_parquet('data/ts_card.parquet')
ts_noninstall = pd.read_parquet('data/ts_noninstall.parquet')

In [3]:
# treat_list = [
#     contract_install,
#     contract_noninstall,
#     contract_card,
#     not_grant_install,
#     not_grant_noninstall,
#     not_grant_card,
#     ts_install,
#     ts_card,
#     ts_noninstall
# ]
# file_name = [
#     'gr_ins', 'gr_nonins', 'gr_card',
#     'ng_ins', 'ng_nonins', 'ng_card',
#     'ts_install', 'ts_card', 'ts_noninstall'
# ]

# def clean_cusid(df):
#     filter_cond = df['id_customer2'].apply(lambda row: len(str(row))).isin([14, 15])
#     return df[filter_cond]

# for i, j in zip(treat_list, file_name):
#     clean_cusid(i).to_parquet(fr'data/{j}.parquet')
    
# for i, j in zip([ts_install, ts_card, ts_noninstall],
#                ['ts_install', 'ts_card', 'ts_noninstall']):
#     i[~i['loan_code_lv2'].isna()].to_parquet(fr'data/{j}.parquet')

<img src="data/data_model.png" alt="data_model" width="400"/>

# II. Generate all features

In [4]:
import importlib
from src_data_process import f_generator
from src_data_process.f_generator import *
importlib.reload(f_generator)
from src_data_process import f_generator
from src_data_process.f_generator import *

In [5]:
from src_data_process.utils import *
from src_data_process.f_generator import *

### 1. Algebric Features

#### Remain exposure

In [6]:
@log_step
def get_remain(root):
    root['total_remain_conso'] = root['Contract.Instalments.ACInstAmounts.RemainingInstalmentsAmount']+\
                        root['Contract.NonInstalments.ACNoInstAmounts.Utilization']+\
                        root['Contract.Cards.ACCardAmounts.ResidualAmount']

    root['tot_limit_conso'] = root['Contract.NonInstalments.ACNoInstAmounts.CreditLimit']+\
                                root['Contract.Cards.ACCardAmounts.LimitOfCredit']
    return root[['id_customer2', 'total_remain_conso', 'tot_limit_conso']]

In [7]:
get_col = ['id_customer2',
 'Contract.Instalments.ACInstAmounts.RemainingInstalmentsAmount',
 'Contract.NonInstalments.ACNoInstAmounts.Utilization',
 'Contract.Cards.ACCardAmounts.ResidualAmount',
 'Contract.NonInstalments.ACNoInstAmounts.CreditLimit',
 'Contract.Cards.ACCardAmounts.LimitOfCredit']

In [8]:
df1 = get_remain(root[get_col])

[get_remain] Shape:(17019, 3). Process time: 0:00:00.000996s


In [9]:
df1.head(5)

,id_customer2,total_remain_conso,tot_limit_conso
0,63926_20241210,36996690.0,47000000.0
0,64875_20241211,3491666.0,1000000.0
0,62215_20241209,2677749.0,17750001.0
0,61225_20241209,10403000.0,0.0
0,65916_20241211,48669256.0,0.0


### 2. Aggregate Features

#### Card

In [10]:
# Extract living card information
field_contract_lv = ['id_customer2', 'loan_code_lv2', 'CreditLimit', 'ResidualAmount', 
                     'MaxNrOfDaysOfPaymentDelay','AmountOverTheLimit', 
                     'WorstStatus', 'DateWorstStatus', 
                     'CommonData.ContractPhase', 'CommonData.StartingDate', 'CommonData.DateOfLastUpdate']
living = contract_card['CommonData.ContractPhase'].isin(['LV'])
card_living = contract_card[living][field_contract_lv]
card_all = contract_card[field_contract_lv]

In [11]:
@log_step
def get_df_card(cus_level, card_data):
    # Join base data to get disbursed date
    df_card = pd.merge(cus_level, card_data, how="left", on=['id_customer2'])
    return df_card
    
# Rule to get updated PCB info (most recent update need to be up to 6-month)
@log_step
def rule_pcb_info(df, fil_col:list):
    """
    Only use report that last updated last 6 months until now
    input column: DateOfLastUpdate (PCB), CreditLimit, ResidualAmount
    """
    df['UpdateDateFmt'] = create_ym_format(df, 'CommonData.DateOfLastUpdate', fmt='%d%m%Y')
    df['mth_snc_last_update'] = df.apply(lambda row: month_diff(row['UpdateDateFmt'], row['created_time']), axis=1)

    df_filtered = df.copy()
    most_update_filter = (df_filtered['mth_snc_last_update'].isna())|(df_filtered['mth_snc_last_update']>6)
    # if contract is last updated in the past >6 months -> not include in calculation
    df_filtered.loc[most_update_filter, fil_col] = np.nan
    return df_filtered

# Aggregate information of contract level -> calculate feature
@log_step
def card_behavior(df):
    """
    cc_ac_ut_rate = sum(CreditLimit)/sum(ResidualAmount)
    """
    aggfunc = {'CreditLimit':'sum',
               'ResidualAmount':'sum',
               'AmountOverTheLimit':['sum', 'mean', 'max'],
               'MaxNrOfDaysOfPaymentDelay':'max'}
    
    df_card_agg = agg_cal(df, groupby=['id_customer2'], aggfunc=aggfunc)
    
    df_card_agg['cc_ac_ut_rate'] = df_card_agg['ResidualAmount_sum']/df_card_agg['CreditLimit_sum']
    df_card_agg['cc_ac_over_lmt'] = df_card_agg['AmountOverTheLimit_sum']/df_card_agg['CreditLimit_sum']
    
    df_card_agg.loc[df_card_agg['CreditLimit_sum']==0, 'cc_ac_ut_rate'] = np.nan
    df_card_agg.loc[df_card_agg['CreditLimit_sum']==0, 'cc_ac_over_lmt'] = np.nan
    
    return df_card_agg

In [12]:
# Processing pipeline
df_card_agg_lv = (cus_level.
               pipe(get_df_card, card_living).
               pipe(rule_pcb_info, ['CreditLimit', 'ResidualAmount']).
               pipe(card_behavior)
              )

[get_df_card] Shape:(24181, 17). Process time: 0:00:00.025914s
[rule_pcb_info] Shape:(24181, 19). Process time: 0:00:00.321922s
[card_behavior] Shape:(16376, 9). Process time: 0:00:00.023919s


In [13]:
df2 = df_card_agg_lv.rename(columns={
    'CreditLimit_sum':'cc_lv_limit',
    'ResidualAmount_sum':'cc_lv_residual',
    'AmountOverTheLimit_sum':'cc_lv_overlmt',
    'AmountOverTheLimit_mean':'cc_lv_overlmt_mean',
    'AmountOverTheLimit_max':'cc_lv_overlmt_max',
    'MaxNrOfDaysOfPaymentDelay_max':'cc_lv_maxDaysOfDelay',
    'cc_ac_ut_rate':'cc_ac_lv_ut_rate',
    'cc_ac_over_lmt':'cc_ac_lv_over_lmt'
})

In [14]:
df2.head(5)

,id_customer2,cc_lv_limit,cc_lv_residual,cc_lv_overlmt,cc_lv_overlmt_mean,cc_lv_overlmt_max,cc_lv_maxDaysOfDelay,cc_ac_lv_ut_rate,cc_ac_lv_over_lmt
0,100005_20241229,20000000.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0
1,100010_20241229,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,100026_20241229,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,100027_20241229,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,100041_20241229,58000000.0,26160598.0,0.0,0.0,0.0,0.0,0.451045,0.0


#### Noninstallment - Overdraft

In [15]:
field_contract_lv = ['id_customer2', 'loan_code_lv2', 'AmountOfTheCredits', 'CommonData.ContractPhase', 'CommonData.StartingDate', 'CommonData.DateOfLastUpdate']
od_living = contract_noninstall['CommonData.ContractPhase'].isin(['LV']) & (contract_noninstall['CommonData.TypeOfFinancing']=='41')
od_all = (contract_noninstall['CommonData.TypeOfFinancing']=='41')

In [16]:
in_lv2 = pd.merge(cus_level, contract_noninstall[od_living][field_contract_lv], 
                  how="left", 
                  on=['id_customer2'])

In [17]:
in_lv3 = pd.merge(cus_level, contract_noninstall[od_all][field_contract_lv], 
                  how="left", 
                  on=['id_customer2'])

In [18]:
od_os = in_lv2[['id_customer2', 'AmountOfTheCredits']]\
                .rename(columns={'AmountOfTheCredits':'lv_od_os'})

In [19]:
od_os_final = od_os.groupby('id_customer2').sum('lv_od_os').reset_index()
od_count = od_os.copy()
od_count = od_count.groupby('id_customer2').count().reset_index().rename(columns={'lv_od_os':'lv_no_od'})
lv_od = pd.merge(od_count, od_os_final, on=['id_customer2'], how='outer')

In [20]:
od_os_all = in_lv3[['id_customer2', 'AmountOfTheCredits']]\
            .rename(columns={'AmountOfTheCredits':'lv_od_os'})

In [21]:
od_os_fn_all = od_os_all.groupby('id_customer2').sum('lv_od_os').reset_index()
od_os_fn_all = od_os_fn_all.rename(columns={'lv_od_os':'od_os'})
od_count_all = od_os_all.copy()
od_count_all = od_count_all.groupby('id_customer2').count().reset_index().rename(columns={'lv_od_os':'no_od'})
od_all = pd.merge(od_count_all, od_os_fn_all, on=['id_customer2'], how='outer')

In [22]:
df3 = pd.merge(lv_od, od_all,
                   how='outer',
                   on='id_customer2')

In [23]:
df3.head(2)

,id_customer2,lv_no_od,lv_od_os,no_od,od_os
0,100005_20241229,1,3500000.0,1,3500000.0
1,100010_20241229,0,0.0,0,0.0


#### Other nonInstallment

In [24]:
contract_noninstall['start_date_fmt'] = contract_noninstall['CommonData.StartingDate']\
            .apply(lambda row: format_start_date(row))
 
contract_noninstall['start_date'] = contract_noninstall['start_date_fmt']\
            .apply(lambda row: get_dt_format(row))

In [25]:
df_nonInst_1st_open = pd.merge(cus_level[['id_customer2', 'created_time']], 
                            contract_noninstall[['id_customer2','start_date']], 
                            how="inner", 
                           on=['id_customer2'])

df_nonInst_1st_open['mth_diff'] = df_nonInst_1st_open.apply(lambda row: month_diff(row['start_date'], row['created_time']), axis=1)

In [26]:
df_nonInst_1st_open = df_nonInst_1st_open[['id_customer2', 'mth_diff']]

In [27]:
df4 = df_nonInst_1st_open\
        .groupby("id_customer2")\
        .agg(nInst_granted_months_from_1st_loan_open=('mth_diff', 'max'))\
        .reset_index()

In [28]:
df4.head()

,id_customer2,nInst_granted_months_from_1st_loan_open
0,100005_20241229,29.0
1,100010_20241229,5.0
2,100026_20241229,NaN
3,100027_20241229,NaN
4,100041_20241229,2.0


In [29]:
len(df4)

16376

#### Installment - Cash/consumer etc. loan

In [30]:
# Count No. of loan by type

In [31]:
living_in = contract_install['CommonData.ContractPhase'].isin(['LV'])
cashLoan_filt = contract_install['CommonData.TypeOfFinancing']=='22'
csLoan_filt = contract_install['CommonData.TypeOfFinancing']=='23'

In [32]:
in_lv2 = contract_install[living_in]

In [33]:
in_lv2 = pd.merge(cus_level, in_lv2, how="left", on=['id_customer2'])

In [34]:
# no_contract_in = rule_pcb_info(in_lv2, fil_col=['CommonData.TypeOfFinancing'])
count_contract = in_lv2[['id_customer2', 'CommonData.TypeOfFinancing']]

In [35]:
cash_loan_count = count_contract[count_contract['CommonData.TypeOfFinancing']=='22'].groupby('id_customer2').count().reset_index()
cash_loan_count.rename(columns={'CommonData.TypeOfFinancing':'no_lv_cashLoan'}, inplace=True)

In [36]:
cs_loan_count = count_contract[count_contract['CommonData.TypeOfFinancing']=='23'].groupby('id_customer2').count().reset_index()
cs_loan_count.rename(columns={'CommonData.TypeOfFinancing':'no_lv_consumerLoan'}, inplace=True)

In [37]:
# Current remaining installment

In [38]:
rename_col={'RemainingInstalmentsAmount':'RemainInstAmt',
            'RemainingInstalmentsNumber':'RemainInstTerm',
           'TotalAmount':'lv_Disbursed_InstAmt'}
           
no_contract_in = in_lv2.rename(columns=rename_col)

In [39]:
remain_amt = agg_cal(no_contract_in,
                       groupby=['id_customer2'],
                       aggfunc={'RemainInstAmt':['sum', 'mean', 'median']}
                    )

remain_term = agg_cal(no_contract_in,
                       groupby=['id_customer2'],
                       aggfunc={'RemainInstTerm':['sum', 'mean', 'median']}
                     )

remain_total = agg_cal(no_contract_in,
                       groupby=['id_customer2'],
                       aggfunc={'lv_Disbursed_InstAmt':['sum', 'mean', 'median']}
                      )

In [40]:
groupAcInst = [cash_loan_count, cs_loan_count, remain_amt, remain_term, remain_total]

df5 = reduce(lambda  
                            left,right: pd.merge(
                                left, right, 
                                on=['id_customer2'], 
                                how='outer'), 
                            groupAcInst)

In [41]:
# final_install_info.to_parquet('feature_store/ins_behav_feature.parquet')

In [42]:
df5.head()

,id_customer2,no_lv_cashLoan,no_lv_consumerLoan,RemainInstAmt_sum,RemainInstAmt_mean,RemainInstAmt_median,RemainInstTerm_sum,RemainInstTerm_mean,RemainInstTerm_median,lv_Disbursed_InstAmt_sum,lv_Disbursed_InstAmt_mean,lv_Disbursed_InstAmt_median
0,100005_20241229,6.0,NaN,28488148.0,4.748025e+06,3106744.5,22.0,3.666667,4.0,32000000.0,5.333333e+06,3500000.0
1,100010_20241229,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
2,100026_20241229,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
3,100027_20241229,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
4,100041_20241229,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN


In [43]:
contract_install['start_date_fmt'] = contract_install['CommonData.StartingDate']\
            .apply(lambda row: format_start_date(row))

contract_install['start_date'] = contract_install['start_date_fmt']\
            .apply(lambda row: get_dt_format(row))


In [44]:
Inst_1st_open = pd.merge(cus_level[['id_customer2', 'created_time']], 
                            contract_install[['id_customer2','start_date']], 
                            how="inner", 
                           on=['id_customer2'])

Inst_1st_open['mth_diff'] = Inst_1st_open.apply(lambda row: month_diff(row['start_date'], row['created_time']), axis=1)

In [45]:
Inst_1st_open = Inst_1st_open\
        .groupby("id_customer2")\
        .agg(inst_granted_months_from_1st_loan_open=('mth_diff', 'max'))\
        .reset_index()

In [46]:
contract_install['LastPaymentDate_fmt'] = contract_install['LastPaymentDate'].apply(
    lambda row: dt.strptime(row, '%d%m%Y') if row != None else None
)
contract_install['mth_snc_last_pmt'] = contract_install['LastPaymentDate_fmt'].apply(
    lambda row: month_diff(row, dt.today())
)

install_last_pmt = contract_install[['id_customer2','mth_snc_last_pmt']]\
                            .groupby('id_customer2')\
                            .agg(inst_months_from_last_pmt=('mth_snc_last_pmt','min'))\
                            .reset_index()

In [47]:
df6 = pd.merge(Inst_1st_open,
                              install_last_pmt,
                              how='outer',
                              on='id_customer2')

In [48]:
# install_since_info.to_parquet('feature_store/inst_snc_feature.parquet')

In [49]:
df6.head(5)

,id_customer2,inst_granted_months_from_1st_loan_open,inst_months_from_last_pmt
0,100005_20241229,34.0,33.0
1,100010_20241229,NaN,NaN
2,100026_20241229,NaN,NaN
3,100027_20241229,NaN,NaN
4,100041_20241229,NaN,NaN


In [50]:
get_col = ['id_customer2', 
           'RemainingInstalmentsAmount', 
           'TotalAmount',
           'RemainingInstalmentsNumber',
           'TotalNumberOfInstalments'
          ]

inst_info_lv = contract_install[living_in][get_col]
inst_info_all = contract_install[living_in][get_col]
consumerLoan_info = contract_install[living_in&csLoan_filt][get_col]

In [51]:
def get_percent_remain(df):
    df = df.groupby('id_customer2').agg(
        remain_amt=('RemainingInstalmentsAmount','sum'),
        remain_term=('RemainingInstalmentsNumber','sum'),
        total_amt=('TotalAmount','sum'),
        total_term=('TotalNumberOfInstalments','sum'),
    ).reset_index()

    df['pct_remain_amt'] = df['remain_amt']/df['total_amt']
    df['pct_remain_term'] = df['remain_term']/df['total_term']
    return df

In [52]:
inst_rm_lv = get_percent_remain(inst_info_lv)
inst_rm_all = get_percent_remain(inst_info_all)
consumerLoan_rm = get_percent_remain(consumerLoan_info)

In [53]:
get_col_rm = ['id_customer2' ,'pct_remain_amt', 'pct_remain_term']

inst_rm_all = inst_rm_all[get_col_rm]
inst_rm_lv = inst_rm_lv[get_col_rm].rename(columns={
    'pct_remain_amt':'pct_rm_amt_lv',
    'pct_remain_term':'pct_rm_term_lv'
})
consumerLoan_rm = consumerLoan_rm[get_col_rm].rename(columns={
    'pct_remain_amt':'pct_rm_amt_consumerL',
    'pct_remain_term':'pct_rm_term_consumerL'
})

In [54]:
df7 = reduce(lambda  left,right: pd.merge(left, right, 
                                                     on=['id_customer2'], 
                                                     how='outer'), 
                        [inst_rm_lv, consumerLoan_rm, inst_rm_all])

In [55]:
df7.head()

,id_customer2,pct_rm_amt_lv,pct_rm_term_lv,pct_rm_amt_consumerL,pct_rm_term_consumerL,pct_remain_amt,pct_remain_term
0,100005_20241229,0.890255,0.814815,NaN,NaN,0.890255,0.814815
1,100048_20241228,1.007107,1.526316,0.976064,0.916667,1.007107,1.526316
2,100060_20241229,0.872349,0.777778,NaN,NaN,0.872349,0.777778
3,100074_20241229,0.874987,0.875000,NaN,NaN,0.874987,0.875000
4,100078_20241228,0.990221,0.656566,NaN,NaN,0.990221,0.656566


#### Overdue

In [56]:
# Extract living card information
field_contract_lv = ['id_customer2', 'loan_code_lv2', 'NrOfDaysOfPaymentDelay', 'CommonData.ContractPhase', 'CommonData.StartingDate', 'CommonData.DateOfLastUpdate']
living_cc = contract_card['CommonData.ContractPhase'].isin(['LV'])
living_in = contract_install['CommonData.ContractPhase'].isin(['LV'])
living_nonin = contract_noninstall['CommonData.ContractPhase'].isin(['LV'])

In [57]:
card_living = contract_card[living_cc][field_contract_lv]
in_living = contract_install[living_in][field_contract_lv]
nonin_living = contract_noninstall[living_nonin][field_contract_lv]

card_lv = pd.merge(cus_level, card_living, how="left", on=['id_customer2'])
in_lv = pd.merge(cus_level, in_living, how="left", on=['id_customer2'])
nonin_lv = pd.merge(cus_level, nonin_living, how="left", on=['id_customer2'])

In [58]:
card_lv.rename(columns={'NrOfDaysOfPaymentDelay':'cc_dpd'}, inplace=True)
in_lv.rename(columns={'NrOfDaysOfPaymentDelay':'in_dpd'}, inplace=True)
nonin_lv.rename(columns={'NrOfDaysOfPaymentDelay':'nonin_dpd'}, inplace=True)

card_lv = card_lv.groupby(['id_customer2']).agg({'cc_dpd':'max'}).reset_index()
in_lv = in_lv.groupby(['id_customer2']).agg({'in_dpd':'max'}).reset_index()
nonin_lv = nonin_lv.groupby(['id_customer2']).agg({'nonin_dpd':'max'}).reset_index()

dpd_lst = [card_lv[['id_customer2', 'cc_dpd']], 
           in_lv[['id_customer2', 'in_dpd']], 
           nonin_lv[['id_customer2', 'nonin_dpd']]
          ]
total_lv = reduce(lambda  left,right: pd.merge(left, right, on='id_customer2', how='outer'), dpd_lst)

In [59]:
total_lv['current_dpd'] = total_lv[['cc_dpd','in_dpd', 'nonin_dpd']].max(axis=1)

In [60]:
df8 = total_lv.groupby('id_customer2').max().reset_index()

In [61]:
df8.head(5)

,id_customer2,cc_dpd,in_dpd,nonin_dpd,current_dpd
0,100005_20241229,0.0,0.0,0.0,0.0
1,100010_20241229,NaN,NaN,0.0,0.0
2,100026_20241229,NaN,NaN,NaN,NaN
3,100027_20241229,NaN,NaN,NaN,NaN
4,100041_20241229,0.0,NaN,0.0,0.0


#### Customer group

In [62]:
non_inst_no_contract_summary = ['Contract.NonInstalments.Summary.NumberOfRequested',
       'Contract.NonInstalments.Summary.NumberOfLiving',
       'Contract.NonInstalments.Summary.NumberOfRefused',
       'Contract.NonInstalments.Summary.NumberOfRenounced',
       'Contract.NonInstalments.Summary.NumberOfTerminated']

inst_no_contract_summary = ['Contract.Instalments.Summary.NumberOfRequested',
       'Contract.Instalments.Summary.NumberOfLiving',
       'Contract.Instalments.Summary.NumberOfRefused',
       'Contract.Instalments.Summary.NumberOfRenounced',
       'Contract.Instalments.Summary.NumberOfTerminated']

card_no_contract_summary = ['Contract.Cards.Summary.NumberOfRequested',
       'Contract.Cards.Summary.NumberOfLiving',
       'Contract.Cards.Summary.NumberOfRefused',
       'Contract.Cards.Summary.NumberOfRenounced',
       'Contract.Cards.Summary.NumberOfTerminated']

rename = ['nonins_req','nonins_lv','nonins_ref','nonins_ren','nonins_ter',
            'ins_req','ins_lv','ins_ref','ins_ren','ins_ter',
            'card_req','card_lv','card_ref','card_ren','card_ter']

all_g = rename.copy()
sum_info = non_inst_no_contract_summary + inst_no_contract_summary + card_no_contract_summary
rename_dict = {}
for old, new in zip(sum_info, rename):
    rename_dict[old] = new

df_suminfo = root[['id_customer2'] + sum_info]
df_suminfo = df_suminfo.rename(columns=rename_dict)

In [63]:
@log_step
def get_df_group(dev_df, df_suminfo):
    # Join base data to get disbursed date
    df_group = pd.merge(dev_df, df_suminfo, how="left", on=['id_customer2'])
    return df_group

In [64]:
@log_step
def agg_group(df_group, all_g):
    matches = ["_lv"]
    group_lv = [i for i in all_g if any(x in i for x in matches)]
    matches = ["_ter"]
    group_tm = [i for i in all_g if any(x in i for x in matches)]
    matches = ["_ref", "_ren", "_req"]
    group_re = [i for i in all_g if any(x in i for x in matches)]
    
    df_group['total_info'] = df_group[all_g].sum(axis=1) # Hit/no hit PCB
    df_group['total_lv'] = df_group[group_lv].sum(axis=1) # Living
    df_group['total_tm'] = df_group[group_tm].sum(axis=1) # Terminated
    df_group['total_re'] = df_group[group_re].sum(axis=1) # Refused + renounced + requested
    return df_group

In [65]:
@log_step
def assign_group_pcb(df_group):
    df_group['flag_hit_pcb'] = np.where(df_group['total_info']>0, 1, 0)
    df_group['group_pcb'] = np.where(
                                        (df_group['total_lv']>0)&(df_group['total_re']+df_group['total_tm']<=0), 'group_2',
                                            np.where(
                                                (df_group['total_lv']+df_group['total_tm']>0)&(df_group['total_re']<=0), 'group_3',
                                                    np.where(
                                                        (df_group['total_lv']+df_group['total_tm']<=0)&(df_group['total_re']>0), 'group_4',
                                                        np.where(df_group['total_info']>0, 'group_1',
                                                                 'group_5'
                                                                )
                                                    )
                                            )
    )
    
    df_group_fn = df_group[['id_customer2','flag_hit_pcb', 'group_pcb']]
    return df_group_fn

In [66]:
df_group = (cus_level.
           pipe(get_df_group, df_suminfo).
           pipe(agg_group, all_g).
           pipe(assign_group_pcb)
           )

[get_df_group] Shape:(16376, 22). Process time: 0:00:00.006977s
[agg_group] Shape:(16376, 26). Process time: 0:00:00.008969s
[assign_group_pcb] Shape:(16376, 3). Process time: 0:00:00.004002s


In [67]:
df9 = df_group[~df_group['id_customer2'].duplicated(keep='first')]

In [68]:
df9.head(5)

,id_customer2,flag_hit_pcb,group_pcb
0,63926_20241210,1,group_1
1,64875_20241211,1,group_1
2,61225_20241209,1,group_1
3,65916_20241211,1,group_1
4,68339_20241212,1,group_1


#### Count FI code (living / all granted)

In [69]:
get = ['id_customer2','CommonData.EncryptedFICode','CommonData.ContractPhase']
all_type = pd.concat([contract_card[get], contract_install[get], contract_noninstall[get]])

In [70]:
def count_unique_FI(c_level, col_name, living=True):
    if living:
        df = c_level[c_level['CommonData.ContractPhase'] == 'LV']
    else:
        df = c_level.copy()
    count_df = df.groupby('id_customer2')['CommonData.EncryptedFICode'].nunique()
    count_df = pd.DataFrame(count_df).reset_index().rename(columns={'CommonData.EncryptedFICode':col_name})
    return count_df

In [71]:
@log_step
def get_unique_FI_no(cus_level):
    df = cus_level[['id_customer2']]
    noFI_all = count_unique_FI(all_type, 'countAll_product', living=False)
    noFI_card = count_unique_FI(contract_card, 'countFI_card', living=False)
    noFI_install = count_unique_FI(contract_install, 'countFI_install', living=False)
    noFI_noinstall = count_unique_FI(contract_noninstall, 'countFI_noninstall', living=False)
    
    noFI_all_lv = count_unique_FI(all_type, 'countAll_product_lv', living=True)
    noFI_card_lv = count_unique_FI(contract_card, 'countFI_card_lv', living=True)
    noFI_install_lv = count_unique_FI(contract_install, 'countFI_install_lv', living=True)
    noFI_noinstall_lv = count_unique_FI(contract_noninstall, 'countFI_noninstall_lv', living=True)
    
    noFI_list = [noFI_all, noFI_card, noFI_install, noFI_noinstall,
                noFI_all_lv, noFI_card_lv, noFI_install_lv, noFI_noinstall_lv]
    for i in noFI_list:
        df = pd.merge(df , i, on=['id_customer2'], how='left')
    return df

In [72]:
df10 = (cus_level
                  .pipe(get_unique_FI_no)
             )

[get_unique_FI_no] Shape:(16376, 9). Process time: 0:00:00.308615s


### 3. Time travel features

In [73]:
ts_card = ts_card[~ts_card['loan_code_lv2'].isna()]
contract_card = contract_card[~contract_card['loan_code_lv2'].isna()]

#### Card

In [74]:
ts_card2 = pd.merge(ts_card, 
                    contract_card[['loan_code_lv2', 'CreditLimit']], 
                    on=['loan_code_lv2'], how='left')
# Aggregate monthly data into customer level
# Ex: Total CC OS per month
ts_card2['ResidualAmount'] = ts_card2['ResidualAmount'].astype(float)
ts_card2['CreditLimit'] = ts_card2['CreditLimit'].astype(float)
ts_card2['Utilization'] = ts_card2['Utilization'].replace('', np.nan)
ts_card2['Utilization'] = ts_card2['Utilization'].astype(float)

In [75]:
card_os = agg_cal(ts_card2,
                       groupby=['id_customer2','ReferenceYear', 'ReferenceMonth'],
                       aggfunc={'ResidualAmount':['sum']}
                 )

card_lmt = agg_cal(ts_card2,
                       groupby=['id_customer2','ReferenceYear', 'ReferenceMonth'],
                       aggfunc={'CreditLimit':['sum']}
                  )

card_utl = agg_cal(ts_card2,
                       groupby=['id_customer2','ReferenceYear', 'ReferenceMonth'],
                       aggfunc={'Utilization':['sum']}
                    )

groupCC = [card_os, card_lmt, card_utl]

card_agg = reduce(lambda  left,right: pd.merge(left, right, on=['id_customer2','ReferenceYear', 'ReferenceMonth'], how='outer'), groupCC)
card_agg['ts_ym'] = card_agg['ReferenceYear'] + card_agg['ReferenceMonth']
card_agg['ts_ym_fmt'] = create_ym_format(card_agg, 'ts_ym', fmt='%Y%m')

In [76]:
df_card = pd.merge(cus_level, card_agg, how="left", on=['id_customer2'])

In [77]:
# Create last X months
# Create input for feature calculation
df_card = pd.merge(cus_level, card_agg, how="left", on=['id_customer2'])
df_card['last_x_months'] = df_card.apply(lambda row: month_diff(row['ts_ym_fmt'], row['created_time']), axis=1)
rename_col = {'ResidualAmount_sum':'cc_os', 
              'CreditLimit_sum':'cc_lmt',
             'Utilization_sum':'cc_utl'
             }
df_card.rename(columns=rename_col, inplace=True)

In [78]:
def ratio(numerator, denominator):
    return np.nan if denominator == 0 else numerator/denominator
    
df_card['cc_utl_rate'] = df_card.apply(lambda row: ratio(row['cc_utl'], row['cc_lmt']), axis=1)
df_card['cc_os_rate'] = df_card.apply(lambda row: ratio(row['cc_os'], row['cc_lmt']), axis=1)

In [79]:
# Generate feature
df11 = generate_feature_lxm(df_card,
                            groupby=['id_customer2'],
                            aggfunc={'cc_os':['mean','max','sum'], 
                                     'cc_lmt':['mean','max','sum'], 
                                     'cc_utl':['mean','max','sum'], 
                                     'cc_os_rate':['mean','max','sum'], 
                                     'cc_utl_rate':['mean','max','sum']},
                            LxM=[3,6,12,24])

In [80]:
df11.head(5)

,id_customer2,cc_os_mean_l4m,cc_os_max_l4m,cc_os_sum_l4m,cc_lmt_mean_l4m,cc_lmt_max_l4m,cc_lmt_sum_l4m,cc_utl_mean_l4m,cc_utl_max_l4m,cc_utl_sum_l4m,cc_os_rate_mean_l4m,cc_os_rate_max_l4m,cc_os_rate_sum_l4m,cc_utl_rate_mean_l4m,cc_utl_rate_max_l4m,...,cc_os_mean_l25m,cc_os_max_l25m,cc_os_sum_l25m,cc_lmt_mean_l25m,cc_lmt_max_l25m,cc_lmt_sum_l25m,cc_utl_mean_l25m,cc_utl_max_l25m,cc_utl_sum_l25m,cc_os_rate_mean_l25m,cc_os_rate_max_l25m,cc_os_rate_sum_l25m,cc_utl_rate_mean_l25m,cc_utl_rate_max_l25m,cc_utl_rate_sum_l25m
0,100005_20241229,1.055377e+05,164010.0,316613.0,20000000.0,20000000.0,60000000.0,0.000000e+00,0.0,0.0,0.005277,0.008200,0.015831,0.00000,0.000000,...,1.651503e+06,3356086.0,23121043.0,2.000000e+07,20000000.0,2.800000e+08,0.000000e+00,0.0,0.0,0.082575,0.167804,1.156052,0.000000,0.000000,0.000000
1,100041_20241229,2.561481e+07,26160598.0,76844427.0,58000000.0,58000000.0,174000000.0,0.000000e+00,0.0,0.0,0.441635,0.451045,1.324904,0.00000,0.000000,...,1.335261e+07,26160598.0,320462550.0,5.425000e+07,58000000.0,1.302000e+09,0.000000e+00,0.0,0.0,0.230996,0.451045,5.543895,0.000000,0.000000,0.000000
2,100060_20241229,4.849268e+06,7011950.0,14547805.0,10000000.0,10000000.0,30000000.0,0.000000e+00,0.0,0.0,0.484927,0.701195,1.454781,0.00000,0.000000,...,1.944503e+06,7011950.0,21389538.0,1.000000e+07,10000000.0,1.100000e+08,0.000000e+00,0.0,0.0,0.194450,0.701195,2.138954,0.000000,0.000000,0.000000
3,100065_20241229,0.000000e+00,0.0,0.0,10000000.0,10000000.0,10000000.0,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000e+00,0.0,0.0,1.000000e+07,10000000.0,1.000000e+07,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,100079_20241229,3.337632e+07,35680562.0,100128970.0,39000000.0,39000000.0,117000000.0,1.221675e+07,23650240.0,36650240.0,0.855803,0.914886,2.567409,0.31325,0.606416,...,2.040449e+07,35680562.0,224449341.0,3.445455e+07,39000000.0,3.790000e+08,6.992422e+06,23650240.0,76916644.0,0.573119,0.914886,6.304305,0.202337,0.717241,2.225709


#### Overdraft

In [81]:
filter_ts = ~ts_noninstall['loan_code_lv2'].isna()
filter_con = ~contract_noninstall['loan_code_lv2'].isna()

In [82]:
# Get overdraft only
ts_noninstall2 = pd.merge(ts_noninstall[filter_ts], 
                          contract_noninstall[filter_con][['loan_code_lv2','CommonData.TypeOfFinancing']], 
                          on=['loan_code_lv2'], how='left')
ts_noninstall2 = ts_noninstall2[ts_noninstall2['CommonData.TypeOfFinancing']=='41']

In [83]:
ts_noninstall2['Granted'] = ts_noninstall2['Granted'].astype(float)

In [84]:
ts_noninstall2['Utilization'] = ts_noninstall2['Utilization'].replace('', np.nan)
ts_noninstall2['Utilization'] = ts_noninstall2['Utilization'].astype(float)

In [85]:
nonInst_lmt = agg_cal(ts_noninstall2,
                       groupby=['id_customer2','ReferenceYear', 'ReferenceMonth'],
                       aggfunc={'Granted':['sum']}
                     )

nonInst_utl = agg_cal(ts_noninstall2,
                       groupby=['id_customer2','ReferenceYear', 'ReferenceMonth'],
                      aggfunc={'Utilization':['sum']}
                     )

nonInstInfo = pd.merge(nonInst_lmt, nonInst_utl, on=['id_customer2','ReferenceYear', 'ReferenceMonth'], how='outer').reset_index()


nonInstInfo['ts_ym'] = nonInstInfo['ReferenceYear'] + nonInstInfo['ReferenceMonth']
nonInstInfo['ts_ym_fmt'] = create_ym_format(nonInstInfo, 'ts_ym', fmt='%Y%m')

In [86]:
# Create last X months
# Create input for feature calculation
df_nonIns = pd.merge(cus_level, nonInstInfo, how="left", on=['id_customer2'])
df_nonIns['last_x_months'] = df_nonIns.apply(lambda row: month_diff(row['ts_ym_fmt'], row['created_time']), axis=1)
df_nonIns.rename(columns={'Granted_sum':'od_lmt'}, inplace=True)
df_nonIns.rename(columns={'Utilization_sum':'od_utl'}, inplace=True)
df_nonIns['od_utl_rate'] = df_nonIns['od_utl']/df_nonIns['od_lmt']

In [87]:
# Generate feature
df12 = generate_feature_lxm(df_nonIns,
                            groupby=['id_customer2'],
                            aggfunc={
                                'od_lmt':['mean','min','max','sum'], 
                                'od_utl':['mean','min','max','sum'], 
                                'od_utl_rate':['mean','min','max','sum']
                            },
                            LxM=[3,6,12,24])

In [88]:
df12.head()

,id_customer2,od_lmt_mean_l4m,od_lmt_min_l4m,od_lmt_max_l4m,od_lmt_sum_l4m,od_utl_mean_l4m,od_utl_min_l4m,od_utl_max_l4m,od_utl_sum_l4m,od_utl_rate_mean_l4m,od_utl_rate_min_l4m,od_utl_rate_max_l4m,od_utl_rate_sum_l4m,od_lmt_mean_l7m,od_lmt_min_l7m,...,od_utl_rate_min_l13m,od_utl_rate_max_l13m,od_utl_rate_sum_l13m,od_lmt_mean_l25m,od_lmt_min_l25m,od_lmt_max_l25m,od_lmt_sum_l25m,od_utl_mean_l25m,od_utl_min_l25m,od_utl_max_l25m,od_utl_sum_l25m,od_utl_rate_mean_l25m,od_utl_rate_min_l25m,od_utl_rate_max_l25m,od_utl_rate_sum_l25m
0,100005_20241229,3500000.0,3500000.0,3500000.0,10500000.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2750000.0,2000000.0,...,0.000000,0.771821,1.926961,1.479167e+06,1000000.0,3500000.0,35500000.0,3.950898e+05,0.0,973361.0,9482154.0,0.353542,0.000000,0.973361,8.485014
1,100048_20241228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,5.000000e+05,0.0,1000000.0,1000000.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,100060_20241229,15825000.0,15825000.0,15825000.0,47475000.0,1.567847e+07,15385417.0,15825000.0,47035417.0,0.990741,0.972222,1.000000,2.972222,15825000.0,15825000.0,...,0.972222,1.000000,2.972222,1.582500e+07,15825000.0,15825000.0,47475000.0,1.567847e+07,15385417.0,15825000.0,47035417.0,0.990741,0.972222,1.000000,2.972222
3,100078_20241228,6000000.0,6000000.0,6000000.0,18000000.0,7.293887e+05,0.0,2188166.0,2188166.0,0.121565,0.000000,0.364694,0.364694,6000000.0,6000000.0,...,0.000000,0.411258,0.775952,5.666667e+06,5000000.0,6000000.0,136000000.0,5.724243e+05,0.0,4912545.0,13738183.0,0.095416,0.000000,0.818758,2.289974
4,100093_20241229,2000000.0,1000000.0,2500000.0,6000000.0,1.484362e+06,959764.0,2496659.0,4453085.0,0.785697,0.398665,0.998664,2.357092,1500000.0,1000000.0,...,0.398665,1.000000,11.137772,1.230769e+06,1000000.0,2500000.0,16000000.0,1.094905e+06,833332.0,2496659.0,14233765.0,0.933675,0.398665,1.000000,12.137772


#### Cash loan, consumer loan

In [89]:
contract_install['ins_mthly_pmt'] = contract_install['TotalAmount']/contract_install['TotalNumberOfInstalments']

In [90]:
filter_ts = ~ts_install['loan_code_lv2'].isna()
filter_con = ~contract_install['loan_code_lv2'].isna()

In [91]:
ts_install2 = pd.merge(ts_install[filter_ts], 
                       contract_install[filter_con][['loan_code_lv2','ins_mthly_pmt', 'CommonData.TypeOfFinancing']],
                       on=['loan_code_lv2'], 
                       how='left')
ts_cashLoan = ts_install2[ts_install2['CommonData.TypeOfFinancing']=='22']
ts_consumerLoan = ts_install2[ts_install2['CommonData.TypeOfFinancing']=='23']

In [92]:
group_col_inst =['id_customer2','ReferenceYear', 'ReferenceMonth']
allInst = agg_cal(ts_install2,
                       groupby=group_col_inst,
                       aggfunc={'ins_mthly_pmt':['sum']}
                 )
cashLoan = agg_cal(ts_cashLoan,
                       groupby=group_col_inst,
                       aggfunc={'ins_mthly_pmt':['sum']}
                 )
consumerLoan = agg_cal(ts_consumerLoan,
                       groupby=group_col_inst,
                       aggfunc={'ins_mthly_pmt':['sum']}
                 )

cashLoan = cashLoan.rename(columns={'ins_mthly_pmt_sum':'cashL_mth_pmt'})
consumerLoan = consumerLoan.rename(columns={'ins_mthly_pmt_sum':'consumerL_mth_pmt'})

InstLst = [allInst, cashLoan, consumerLoan]

inst_lmt = reduce(lambda  left,right: pd.merge(left, right, on=group_col_inst, how='outer'), InstLst)
inst_lmt['ts_ym'] = inst_lmt['ReferenceYear'] + inst_lmt['ReferenceMonth']
inst_lmt['ts_ym_fmt'] = create_ym_format(inst_lmt, 'ts_ym', fmt='%Y%m')

In [93]:
# Create last X months
# Create input for feature calculation
df_Ins = pd.merge(cus_level, inst_lmt, how="left", on=['id_customer2'])
df_Ins['last_x_months'] = df_Ins.apply(lambda row: month_diff(row['ts_ym_fmt'], row['created_time']), axis=1)
df_Ins.rename(columns={'ins_mthly_pmt_sum':'ins_mthly_pmt'}, inplace=True)

In [94]:
df13 = generate_feature_lxm(df_Ins,
                            groupby=['id_customer2'],
                            aggfunc={
                                'ins_mthly_pmt':['mean','max','sum'], 
                                'cashL_mth_pmt':['mean','max','sum'], 
                                'consumerL_mth_pmt':['mean','max','sum']
                            },
                            LxM=[3,6,12,24])

In [95]:
df13.head(5)

,id_customer2,ins_mthly_pmt_mean_l4m,ins_mthly_pmt_max_l4m,ins_mthly_pmt_sum_l4m,cashL_mth_pmt_mean_l4m,cashL_mth_pmt_max_l4m,cashL_mth_pmt_sum_l4m,consumerL_mth_pmt_mean_l4m,consumerL_mth_pmt_max_l4m,consumerL_mth_pmt_sum_l4m,ins_mthly_pmt_mean_l7m,ins_mthly_pmt_max_l7m,ins_mthly_pmt_sum_l7m,cashL_mth_pmt_mean_l7m,cashL_mth_pmt_max_l7m,...,cashL_mth_pmt_mean_l13m,cashL_mth_pmt_max_l13m,cashL_mth_pmt_sum_l13m,consumerL_mth_pmt_mean_l13m,consumerL_mth_pmt_max_l13m,consumerL_mth_pmt_sum_l13m,ins_mthly_pmt_mean_l25m,ins_mthly_pmt_max_l25m,ins_mthly_pmt_sum_l25m,cashL_mth_pmt_mean_l25m,cashL_mth_pmt_max_l25m,cashL_mth_pmt_sum_l25m,consumerL_mth_pmt_mean_l25m,consumerL_mth_pmt_max_l25m,consumerL_mth_pmt_sum_l25m
0,100005_20241229,5.833333e+06,6.166667e+06,1.750000e+07,5.833333e+06,6.166667e+06,1.750000e+07,NaN,NaN,0.000000e+00,3.750000e+06,6.166667e+06,2.250000e+07,3.750000e+06,6.166667e+06,...,2.772727e+06,6.166667e+06,3.050000e+07,NaN,NaN,0.000000e+00,2.772727e+06,6.166667e+06,3.050000e+07,2.772727e+06,6.166667e+06,3.050000e+07,NaN,NaN,0.000000e+00
1,100048_20241228,7.168646e+06,1.194865e+07,1.433729e+07,5.780000e+06,1.056000e+07,1.156000e+07,1.388646e+06,1.388646e+06,2.777292e+06,7.168646e+06,1.194865e+07,1.433729e+07,5.780000e+06,1.056000e+07,...,5.780000e+06,1.056000e+07,1.156000e+07,1.388646e+06,1.388646e+06,2.777292e+06,7.168646e+06,1.194865e+07,1.433729e+07,5.780000e+06,1.056000e+07,1.156000e+07,1.388646e+06,1.388646e+06,2.777292e+06
2,100060_20241229,1.055556e+06,1.277778e+06,3.166667e+06,1.055556e+06,1.277778e+06,3.166667e+06,NaN,NaN,0.000000e+00,1.055556e+06,1.277778e+06,3.166667e+06,1.055556e+06,1.277778e+06,...,1.055556e+06,1.277778e+06,3.166667e+06,NaN,NaN,0.000000e+00,1.055556e+06,1.277778e+06,3.166667e+06,1.055556e+06,1.277778e+06,3.166667e+06,NaN,NaN,0.000000e+00
3,100074_20241229,9.326250e+05,9.326250e+05,1.865250e+06,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,9.326250e+05,9.326250e+05,1.865250e+06,NaN,NaN,...,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,9.618214e+05,9.735000e+05,6.732750e+06,NaN,NaN,0.000000e+00,9.735000e+05,9.735000e+05,4.867500e+06
4,100078_20241228,7.173722e+06,7.632278e+06,2.152117e+07,6.611111e+06,6.944444e+06,1.983333e+07,NaN,NaN,0.000000e+00,6.179454e+06,7.632278e+06,3.707672e+07,5.898148e+06,6.944444e+06,...,3.615741e+06,6.944444e+06,4.338889e+07,1.485750e+06,1.485750e+06,2.971500e+06,5.072559e+06,7.887283e+06,1.217414e+08,4.630795e+06,6.944444e+06,1.111391e+08,1.485750e+06,1.485750e+06,8.914500e+06


#### Overdue

In [96]:
# Total number of overdue term per month across multiple contracts
group_col = ['id_customer2','ReferenceYear', 'ReferenceMonth']
ts_card['Default'] = ts_card['Default'].astype(float).fillna(0)
ts_install['Default'] = ts_install['Default'].astype(float).fillna(0)

ccOverdue = agg_cal(ts_card,
                       groupby=group_col,
                       aggfunc={'Default':['sum']}
                   )

InsOverdue = agg_cal(ts_install,
                     groupby=group_col,
                     aggfunc={'Default':['sum']}
                   )

ccOverdue.rename(columns={'Default_sum':'cc_od_term'}, inplace=True)
InsOverdue.rename(columns={'Default_sum':'inst_od_term'}, inplace=True)

In [97]:
dfOverdueTerm = pd.merge(ccOverdue, InsOverdue, on=group_col, how='outer')
dfOverdueTerm['total_od_term'] = dfOverdueTerm['cc_od_term']+dfOverdueTerm['inst_od_term']

In [98]:
# Create last X months
# Create input for feature calculation
df_od = pd.merge(cus_level[['id_customer2', 'created_time']], dfOverdueTerm, how="left", on=['id_customer2'])
df_od['ts_ym'] = df_od['ReferenceYear'] + df_od['ReferenceMonth']
df_od['ts_ym_fmt'] = create_ym_format(df_od, 'ts_ym', fmt='%Y%m')
df_od['last_x_months'] = df_od.apply(lambda row: month_diff(row['ts_ym_fmt'], row['created_time']), axis=1)

In [99]:
# Generate feature
df14 = generate_feature_lxm(df_od,
                            groupby=['id_customer2'],
                            aggfunc={
                                'cc_od_term':['mean','max'], 
                                'inst_od_term':['mean','max']   
                            },
                            LxM=[3,6,12,24])

In [100]:
df14.head(5)

,id_customer2,cc_od_term_mean_l4m,cc_od_term_max_l4m,inst_od_term_mean_l4m,inst_od_term_max_l4m,cc_od_term_mean_l7m,cc_od_term_max_l7m,inst_od_term_mean_l7m,inst_od_term_max_l7m,cc_od_term_mean_l13m,cc_od_term_max_l13m,inst_od_term_mean_l13m,inst_od_term_max_l13m,cc_od_term_mean_l25m,cc_od_term_max_l25m,inst_od_term_mean_l25m,inst_od_term_max_l25m
0,100005_20241229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100041_20241229,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
2,100048_20241228,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0
3,100060_20241229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100065_20241229,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [101]:
# Total number of overdue term per month across multiple contracts
group_col = ['id_customer2','ReferenceYear', 'ReferenceMonth']
ts_card.replace('', np.nan, inplace=True)
ts_install.replace('', np.nan, inplace=True)
ts_noninstall.replace('', np.nan, inplace=True)

ts_card['Status'] = ts_card['Status'].astype(float)
ts_install['Status'] = ts_install['Status'].astype(float)
ts_noninstall['Status'] = ts_noninstall['Status'].astype(float)

ccStatus = agg_cal(ts_card,
                       groupby=group_col,
                       aggfunc={'Status':['max']}
                  )

InsStatus = agg_cal(ts_install,
                       groupby=group_col,
                       aggfunc={'Status':['max']}
                  )

NonInsStatus = agg_cal(ts_noninstall,
                       groupby=group_col,
                       aggfunc={'Status':['max']}
                  )

ccStatus.rename(columns={'Status_max':'cc_ln_grp'}, inplace=True)
InsStatus.rename(columns={'Status_max':'in_ln_grp'}, inplace=True)
NonInsStatus.rename(columns={'Status_max':'nonin_ln_group'}, inplace=True)

groupSt = [ccStatus, InsStatus, NonInsStatus]
totalStatus = reduce(lambda  left,right: pd.merge(left, right, on=group_col, how='outer'), groupSt)

In [102]:
# Create last X months
# Create input for feature calculation
dfStatus = pd.merge(cus_level[['id_customer2', 'created_time']], totalStatus, how="left", on=['id_customer2'])
dfStatus['ts_ym'] = dfStatus['ReferenceYear'] + dfStatus['ReferenceMonth']
dfStatus['ts_ym_fmt'] = create_ym_format(dfStatus, 'ts_ym', fmt='%Y%m')
dfStatus['last_x_months'] = dfStatus.apply(lambda row: month_diff(row['ts_ym_fmt'], row['created_time']), axis=1)

In [103]:
dfStatus['ln_grp'] = dfStatus[['cc_ln_grp', 'in_ln_grp','nonin_ln_group']].max(axis=1)

In [104]:
# Generate feature
df15 = generate_feature_lxm(dfStatus,
                            groupby=['id_customer2'],
                            aggfunc={'cc_ln_grp':['max'], 
                                     'in_ln_grp':['max'], 
                                     'nonin_ln_group':['max'], 
                                     'ln_grp':['max']},
                            LxM=[3,6,12,24])

In [105]:
df15.head()

,id_customer2,cc_ln_grp_max_l4m,in_ln_grp_max_l4m,nonin_ln_group_max_l4m,ln_grp_max_l4m,cc_ln_grp_max_l7m,in_ln_grp_max_l7m,nonin_ln_group_max_l7m,ln_grp_max_l7m,cc_ln_grp_max_l13m,in_ln_grp_max_l13m,nonin_ln_group_max_l13m,ln_grp_max_l13m,cc_ln_grp_max_l25m,in_ln_grp_max_l25m,nonin_ln_group_max_l25m,ln_grp_max_l25m
0,100005_20241229,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,100010_20241229,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0
2,100041_20241229,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0
3,100048_20241228,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0
4,100060_20241229,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [106]:
def get_mth_since_status(df):
    df['DateWorstStatus_fmt'] = df['DateWorstStatus']\
                .apply(
                    lambda row: dt.strptime(row,'%d%m%Y') if row != None else None
                )

    since_status = df[~df['DateWorstStatus_fmt'].isna()][['id_customer2',
                                                                                   'DateWorstStatus_fmt', 
                                                                                   'WorstStatus']]
    
    since_status['mth_snc_worstSt'] = since_status['DateWorstStatus_fmt'].apply(
                                                lambda row: month_diff(row, dt.today())
    )
    return since_status

In [107]:
def since_last_status(df):
    snc_last_active = df[df['WorstStatus']==1]\
                        .groupby('id_customer2')\
                        .agg(
                            mth_snc_lst_active = ('mth_snc_worstSt','min')
                        ).reset_index()
    
    snc_last_overdue = df[df['WorstStatus']!=1]\
                        .groupby('id_customer2')\
                        .agg(
                            mth_snc_lst_overdue = ('mth_snc_worstSt','min')
                        ).reset_index()
    
    Since_last_status = pd.merge(snc_last_active, 
                         snc_last_overdue,
                         on='id_customer2',
                         how='outer')
    return Since_last_status

In [108]:
since_status_i  = get_mth_since_status(contract_install)
since_status_ni = get_mth_since_status(contract_noninstall)
since_status_c  = get_mth_since_status(contract_card)

since_st_i  = since_last_status(since_status_i)
since_st_ni = since_last_status(since_status_ni)
since_st_c  = since_last_status(since_status_c)

In [109]:
since_st_i = since_st_i.rename(columns={
    'mth_snc_lst_active':'inst_mth_snc_lst_active',
    'mth_snc_lst_overdue':'inst_mth_snc_lst_overdue'
})

since_st_ni = since_st_ni.rename(columns={
    'mth_snc_lst_active':'nonInst_mth_snc_lst_active',
    'mth_snc_lst_overdue':'nonInst_mth_snc_lst_overdue'
})

since_st_c = since_st_c.rename(columns={
    'mth_snc_lst_active':'card_mth_snc_lst_active',
    'mth_snc_lst_overdue':'card_mth_snc_lst_overdue'
})              

In [110]:
df16 = reduce(lambda  left,right: pd.merge(left, right, 
                                                     on=['id_customer2'], 
                                                     how='outer'), 
                        [since_st_i, since_st_ni, since_st_c])

In [111]:
df16.head(5)

,id_customer2,inst_mth_snc_lst_active,inst_mth_snc_lst_overdue,nonInst_mth_snc_lst_active,nonInst_mth_snc_lst_overdue,card_mth_snc_lst_active,card_mth_snc_lst_overdue
0,100005_20241229,5.0,NaN,5.0,NaN,5.0,NaN
1,100010_20241229,NaN,NaN,5.0,NaN,NaN,NaN
2,100041_20241229,NaN,NaN,5.0,NaN,5.0,NaN
3,100048_20241228,5.0,NaN,5.0,NaN,NaN,NaN
4,100060_20241229,5.0,NaN,5.0,NaN,5.0,NaN


In [112]:
inst_mths_d = contract_install[contract_install['MaximumLevelOfDefault']>0]\
    .groupby('id_customer2')\
    .agg(inst_mths_indefault = ('NumberOfMonthsWithMaximumLevelOfDefault','max'))

noninst_mths_d = contract_noninstall[contract_noninstall['MaximumLevelOfDefault']>0]\
    .groupby('id_customer2')\
    .agg(noninst_mths_indefault = ('NumberOfMonthsWithMaximumLevelOfDefault','max'))

card_mths_d = contract_card[contract_card['MaximumLevelOfDefault']>0]\
    .groupby('id_customer2')\
    .agg(card_mths_indefault = ('NumberOfMonthsWithMaximumLevelOfDefault','max'))

df17 = reduce(lambda  left,right: pd.merge(left, right, 
                                                     on=['id_customer2'], 
                                                     how='outer'), 
                        [inst_mths_d, noninst_mths_d, card_mths_d]).reset_index()

df17['total_mths_indefault'] = df17[['inst_mths_indefault',
                                                         'noninst_mths_indefault',	
                                                         'card_mths_indefault']].sum(axis=1)

In [113]:
df17.head(5)

,id_customer2,inst_mths_indefault,noninst_mths_indefault,card_mths_indefault,total_mths_indefault
0,100093_20241229,NaN,NaN,1.0,1.0
1,100147_20241229,NaN,NaN,5.0,5.0
2,100147_20250128,NaN,NaN,5.0,5.0
3,100147_20250213,NaN,NaN,5.0,5.0
4,100155_20241229,NaN,NaN,1.0,1.0


### 4. Other features

In [114]:
st_type = ['Refused','Renounced','Requested']
get_lst = []
for i in root.columns:
    if any(t in i for t in st_type):
        get_lst.append(i)

In [115]:
df_notGrant = root[['id_customer2']+get_lst].copy()

In [116]:
df_notGrant['contracts_summary_not_granted_contracts'] = df_notGrant[get_lst].sum(axis=1)

In [117]:
# Instalment not grant

In [118]:
df_ins_notGrant = not_grant_install.groupby('id_customer2')\
                        .agg({'Amounts.TotalNumberOfInstalments':'mean'})\
                        .reset_index()

In [119]:
df_ins_notGrant.columns = ['id_customer2', 'instalments_not_granted_total_instalments_avg']

In [120]:
# Card summary renouces

In [121]:
df_card_rn = root[['id_customer2', 'Contract.Cards.Summary.NumberOfRenounced']].copy()
df_card_rn.columns = ['id_customer2','card_summary_renounces']

In [122]:
df18 = reduce(lambda left,right: pd.merge(left, right,
                          how='outer', on='id_customer2'),
    [df_notGrant, df_ins_notGrant, df_card_rn]).reset_index(drop=True)

In [123]:
df18.head(5)

,id_customer2,Contract.Instalments.Summary.NumberOfRequested,Contract.Instalments.Summary.NumberOfRefused,Contract.Instalments.Summary.NumberOfRenounced,Contract.NonInstalments.Summary.NumberOfRequested,Contract.NonInstalments.Summary.NumberOfRefused,Contract.NonInstalments.Summary.NumberOfRenounced,Contract.Cards.Summary.NumberOfRequested,Contract.Cards.Summary.NumberOfRefused,Contract.Cards.Summary.NumberOfRenounced,contracts_summary_not_granted_contracts,instalments_not_granted_total_instalments_avg,card_summary_renounces
0,100005_20241229,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,18.0,0.0
1,100010_20241229,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,0.0
2,100026_20241229,4.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,6.0,21.0,0.0
3,100027_20241229,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,3.5,0.0
4,100041_20241229,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,15.2,0.0


In [124]:
# cards_granted_months_from_1st_loan_open

In [125]:
contract_card['start_date_fmt'] = contract_card['CommonData.StartingDate']\
            .apply(lambda row: format_start_date(row))

contract_card['start_date'] = contract_card['start_date_fmt']\
            .apply(lambda row: get_dt_format(row))

In [126]:
df_card_1st_open = pd.merge(cus_level[['id_customer2', 'created_time']], 
                            contract_card[['id_customer2','start_date']], 
                            how="inner", 
                           on=['id_customer2'])

df_card_1st_open['mth_diff'] = df_card_1st_open.apply(lambda row: month_diff(row['start_date'], row['created_time']), axis=1)

In [127]:
df19 = df_card_1st_open\
        .groupby("id_customer2")\
        .agg(cards_granted_months_from_1st_loan_open=('mth_diff', 'max'))\
        .reset_index()

In [128]:
df19.head(5)

,id_customer2,cards_granted_months_from_1st_loan_open
0,100005_20241229,17
1,100041_20241229,18
2,100060_20241229,12
3,100065_20241229,2
4,100079_20241229,12


In [129]:
# contracts_summary_terminates

In [130]:
tm_col = []
for i in root.columns:
    if 'Terminated' in i:
        tm_col.append(i)

In [131]:
df20 = root[['id_customer2']+tm_col].copy()

In [132]:
df20['contracts_summary_terminates'] = df20[tm_col].sum(axis=1)

In [133]:
df20.head(5)

,id_customer2,Contract.Instalments.Summary.NumberOfTerminated,Contract.NonInstalments.Summary.NumberOfTerminated,Contract.Cards.Summary.NumberOfTerminated,contracts_summary_terminates
0,63926_20241210,3.0,0.0,0.0,3.0
0,64875_20241211,0.0,0.0,0.0,0.0
0,62215_20241209,1.0,0.0,1.0,2.0
0,61225_20241209,1.0,0.0,0.0,1.0
0,65916_20241211,1.0,0.0,1.0,2.0


# III. Console, calculate score

In [134]:
conso_feat = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10,
        df11, df12, df13, df14, df15, df16, df17, df18, df19, df20]

In [149]:
final_merge_df = reduce(lambda  left,right: pd.merge(left, right, 
                                                     on=['id_customer2'], 
                                                     how='outer'), 
                        conso_feat)

In [ ]:
final_merge_df

In [145]:
final_merge_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17019 entries, 0 to 17018
Data columns (total 252 columns):
 #    Column                                              Non-Null Count  Dtype  
---   ------                                              --------------  -----  
 0    id_customer2                                        17019 non-null  object 
 1    total_remain_conso                                  17019 non-null  float64
 2    tot_limit_conso                                     17019 non-null  float64
 3    cc_lv_limit                                         16376 non-null  float64
 4    cc_lv_residual                                      16376 non-null  float64
 5    cc_lv_overlmt                                       16376 non-null  float64
 6    cc_lv_overlmt_mean                                  7421 non-null   float64
 7    cc_lv_overlmt_max                                   7421 non-null   float64
 8    cc_lv_maxDaysOfDelay                                7352 non-nul

In [146]:
search_term = ['cc_', 'od_', 
              'cashL_', 'consumerL_']
[i for i in final_merge_df.columns if any(term in i for term in search_term)]

['cc_lv_limit',
 'cc_lv_residual',
 'cc_lv_overlmt',
 'cc_lv_overlmt_mean',
 'cc_lv_overlmt_max',
 'cc_lv_maxDaysOfDelay',
 'cc_ac_lv_ut_rate',
 'cc_ac_lv_over_lmt',
 'lv_od_os',
 'od_os',
 'cc_dpd',
 'cc_os_mean_l4m',
 'cc_os_max_l4m',
 'cc_os_sum_l4m',
 'cc_lmt_mean_l4m',
 'cc_lmt_max_l4m',
 'cc_lmt_sum_l4m',
 'cc_utl_mean_l4m',
 'cc_utl_max_l4m',
 'cc_utl_sum_l4m',
 'cc_os_rate_mean_l4m',
 'cc_os_rate_max_l4m',
 'cc_os_rate_sum_l4m',
 'cc_utl_rate_mean_l4m',
 'cc_utl_rate_max_l4m',
 'cc_utl_rate_sum_l4m',
 'cc_os_mean_l7m',
 'cc_os_max_l7m',
 'cc_os_sum_l7m',
 'cc_lmt_mean_l7m',
 'cc_lmt_max_l7m',
 'cc_lmt_sum_l7m',
 'cc_utl_mean_l7m',
 'cc_utl_max_l7m',
 'cc_utl_sum_l7m',
 'cc_os_rate_mean_l7m',
 'cc_os_rate_max_l7m',
 'cc_os_rate_sum_l7m',
 'cc_utl_rate_mean_l7m',
 'cc_utl_rate_max_l7m',
 'cc_utl_rate_sum_l7m',
 'cc_os_mean_l13m',
 'cc_os_max_l13m',
 'cc_os_sum_l13m',
 'cc_lmt_mean_l13m',
 'cc_lmt_max_l13m',
 'cc_lmt_sum_l13m',
 'cc_utl_mean_l13m',
 'cc_utl_max_l13m',
 'cc_utl_sum

In [138]:
final_merge_df.head()

,id_customer2,total_remain_conso,tot_limit_conso,cc_lv_limit,cc_lv_residual,cc_lv_overlmt,cc_lv_overlmt_mean,cc_lv_overlmt_max,cc_lv_maxDaysOfDelay,cc_ac_lv_ut_rate,cc_ac_lv_over_lmt,lv_no_od,lv_od_os,no_od,od_os,...,Contract.Instalments.Summary.NumberOfRenounced,Contract.NonInstalments.Summary.NumberOfRequested,Contract.NonInstalments.Summary.NumberOfRefused,Contract.NonInstalments.Summary.NumberOfRenounced,Contract.Cards.Summary.NumberOfRequested,Contract.Cards.Summary.NumberOfRefused,Contract.Cards.Summary.NumberOfRenounced,contracts_summary_not_granted_contracts,instalments_not_granted_total_instalments_avg,card_summary_renounces,cards_granted_months_from_1st_loan_open,Contract.Instalments.Summary.NumberOfTerminated,Contract.NonInstalments.Summary.NumberOfTerminated,Contract.Cards.Summary.NumberOfTerminated,contracts_summary_terminates
0,100005_20241229,28488148.0,23500000.0,20000000.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,1.0,3500000.0,1.0,3500000.0,...,1.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,18.0,0.0,17.0,3.0,0.0,0.0,3.0
1,100010_20241229,1418911.0,2000000.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0
2,100026_20241229,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,6.0,21.0,0.0,NaN,0.0,0.0,0.0,0.0
3,100027_20241229,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,3.5,0.0,NaN,0.0,0.0,0.0,0.0
4,100041_20241229,26633873.0,61000000.0,58000000.0,26160598.0,0.0,0.0,0.0,0.0,0.451045,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,15.2,0.0,18.0,0.0,0.0,0.0,0.0


In [ ]:
label

In [193]:
def get_label(row,threshold=5):
    if row >= threshold:
        return 1
    else:
        return 0

In [194]:
label['label'] = label['Fpd'].apply(lambda row: get_label(row))

In [195]:
label['label'].value_counts(dropna=False)

label
0    238197
1     10972
Name: count, dtype: int64

In [196]:
label

,id_customer,id_contract,utm_source,disbursed_date,id_customer2,Fpd,Spd,Tpd,maxdpd_n4m,maxdpd_n5m,maxdpd_n6m,1due,2due,3due,4due,5due,6due,label_fpd,label
0,180893,VC10277136658,MOMO,2025-02-05,180893_20250205,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0
1,181443,VC10730914485,MOMO,2025-02-06,181443_20250206,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0
2,181679,VC14907269439,MOMO,2025-02-06,181679_20250206,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0
3,67812,VM12122917176,VTMN,2024-12-12,67812_20241212,0,0,0,0,0,0,1,1,0,0,0,0,0,0
4,67994,VM12128076329,VTMN,2024-12-12,67994_20241212,0,0,0,0,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264118,38275,VC19138567232,MOMO,2024-11-24,38275_20241124,0,0,0,0,0,0,1,1,1,0,0,0,0,0
264119,215120,VC11758561269,MOMO,2025-02-28,215120_20250228,-37,-37,-37,-37,-37,-37,0,0,0,0,0,0,0,0
264120,54793,VC18243502217,MOMO,2024-12-05,54793_20241205,0,0,0,0,0,0,1,1,0,0,0,0,0,0
264121,87281,VC13635676893,MOMO,2024-12-22,87281_20241222,0,0,0,0,0,0,1,1,0,0,0,0,0,0


In [197]:
to_export = pd.merge(final_merge_df, 
         label[['id_customer2', 
                'label', 'disbursed_date']]
        )

In [198]:
# to_export['disbursed_date'] = to_export['disbursed_date'].apply(lambda row: dt.strptime(row, '%Y-%m-%d'))

In [199]:
to_export.to_parquet('data/train_data.parquet')